In [1]:
from yugiquery import *
header('Speed Duel')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Speed Duel - Execution started 11/02/2023 17:49 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

In [2]:
skill_df = fetch_skill()

https://yugipedia.com/api.php?action=ask&format=json&query=[[Category:Skill%20Cards]]|?English%20name=Name|?Card%20type|?Archseries|?TCG%20status|?TCG%20Speed%20Duel%20status|?Modification%20date|limit%3D5000|offset=0|order%3Dasc
Iteration 1: 105 results
105 results



In [3]:
skill_df

,Name,Card type,Archseries,TCG status,TCG Speed Duel status,Page name,Page URL,Modification date
0,Ancient Fusion,Skill Card,"(Fusion (archetype),)",Illegal,Legal,Ancient Fusion,https://yugipedia.com/wiki/Ancient_Fusion,2022-12-06 13:13:12
1,Armed and Ready!,Skill Card,NaN,Illegal,Legal,Armed and Ready!,https://yugipedia.com/wiki/Armed_and_Ready!,2022-09-01 17:48:53
2,Aroma Strategy,Skill Card,NaN,Illegal,Legal,Aroma Strategy,https://yugipedia.com/wiki/Aroma_Strategy,2023-01-02 10:39:59
3,Bandit,Skill Card,NaN,Illegal,Legal,Bandit,https://yugipedia.com/wiki/Bandit,2020-12-02 03:33:23
4,Beasts of Phantom,Skill Card,NaN,Illegal,Legal,Beasts of Phantom,https://yugipedia.com/wiki/Beasts_of_Phantom,2022-12-27 18:47:29
...,...,...,...,...,...,...,...,...
100,Union Combination,Skill Card,NaN,Illegal,Legal,Union Combination,https://yugipedia.com/wiki/Union_Combination,2023-02-09 12:07:11
101,Viral Infection,Skill Card,NaN,Illegal,Legal,Viral Infection,https://yugipedia.com/wiki/Viral_Infection,2020-05-29 17:27:57
102,Volcanic Cannon,Skill Card,"(Volcanic,)",Illegal,Legal,Volcanic Cannon,https://yugipedia.com/wiki/Volcanic_Cannon,2022-09-01 17:16:50
103,The World's Greatest Fisherman,Skill Card,NaN,Illegal,Legal,The World's Greatest Fisherman,https://yugipedia.com/wiki/The_World%27s_Great...,2020-05-29 17:49:58


In [4]:
footer()

<div align='right'>
    Execution completed 11/02/2023 17:49 UTC
</div>

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Speed.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Ss]peed*"

In [ ]:
! git commit -m {"'Speed duel update-" + timestamp.isoformat() + "'"}